In [1]:
import pandas as pd

# Load SGV data
sgv_data = pd.read_csv("/Users/yunruyeh/Desktop/W9_health_project/original_sgv.csv", sep=';')

# Load Treatments data
treatments_data = pd.read_excel("/Users/yunruyeh/Desktop/W9_health_project/original_treatments_mongoDB.xlsx", sheet_name='treatments')





In [3]:
# Parse timestamps and select relevant columns for SGV
sgv_data['timestamp'] = pd.to_datetime(sgv_data['dateString'], errors='coerce', utc=True)
sgv_cleaned = sgv_data[['timestamp', 'sgv']].dropna()

# Parse timestamps and select relevant columns for treatments
treatments_data['timestamp'] = pd.to_datetime(treatments_data['created_at'], errors='coerce', utc=True)
treatments_cleaned = treatments_data[['timestamp', 'carbs', 'insulin']].dropna(subset=['timestamp'])

/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/3594396217.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sgv_data['timestamp'] = pd.to_datetime(sgv_data['dateString'], errors='coerce', utc=True)


## Resample blood glucose data (SGV) every 5 minutes:
Use Pandas's resample() method to sample and calculate the average

In [5]:
sgv_cleaned = sgv_cleaned.set_index('timestamp')

# Resample SGV every 5 minutes
sgv_resampled = sgv_cleaned['sgv'].resample('5T').mean().reset_index()

print(sgv_resampled.head())

                  timestamp    sgv
0 2019-06-07 12:45:00+00:00  230.0
1 2019-06-07 12:50:00+00:00  227.0
2 2019-06-07 12:55:00+00:00  235.0
3 2019-06-07 13:00:00+00:00  252.0
4 2019-06-07 13:05:00+00:00  257.0


/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/3606396640.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  sgv_resampled = sgv_cleaned['sgv'].resample('5T').mean().reset_index()


In [8]:
sgv_resampled

,timestamp,sgv
0,2019-06-07 12:45:00+00:00,230.0
1,2019-06-07 12:50:00+00:00,227.0
2,2019-06-07 12:55:00+00:00,235.0
3,2019-06-07 13:00:00+00:00,252.0
4,2019-06-07 13:05:00+00:00,257.0
...,...,...
32520,2019-09-28 10:45:00+00:00,107.0
32521,2019-09-28 10:50:00+00:00,109.0
32522,2019-09-28 10:55:00+00:00,112.0
32523,2019-09-28 11:00:00+00:00,117.0


In [10]:
treatments_cleaned

,timestamp,carbs,insulin
0,2019-05-07 13:40:08+00:00,NaN,1.0
1,2019-05-03 11:18:45+00:00,NaN,0.0
2,2019-05-16 09:22:29+00:00,NaN,NaN
3,2019-05-15 22:08:15+00:00,NaN,NaN
4,2019-05-03 12:13:35+00:00,NaN,NaN
...,...,...,...
25465,2019-09-21 11:59:21+00:00,NaN,NaN
25466,2019-09-21 12:08:07+00:00,NaN,NaN
25467,2019-09-21 12:18:18+00:00,NaN,NaN
25468,2019-09-21 12:28:02+00:00,NaN,NaN


In [12]:
print(sgv_data.columns)
print(treatments_cleaned.columns)

Index(['date', 'dateString', 'direction', 'sgv', 'type', 'timestamp'], dtype='object')
Index(['timestamp', 'carbs', 'insulin'], dtype='object')


## Keep original insulin and carbohydrate data

In [15]:
# Step 1: Resample SGV data (assuming 'timestamp' is the time column)
sgv_resampled = sgv_data[['timestamp', 'sgv']].sort_values(by='timestamp').reset_index(drop=True)

# Step 2: Extract and sort insulin and carbs data
insulin_carbs = treatments_cleaned[['timestamp', 'insulin', 'carbs']].sort_values(by='timestamp').reset_index(drop=True)

# Step 3: Use merge_asof for time alignment
result = pd.merge_asof(sgv_resampled, insulin_carbs, on='timestamp')

# Step 4: Optional: Sort the final result again (if needed)
result.sort_values(by='timestamp', inplace=True)

# Display result
result

,timestamp,sgv,insulin,carbs
0,2019-06-07 12:49:41+00:00,230,NaN,97.5
1,2019-06-07 12:54:41+00:00,227,NaN,97.5
2,2019-06-07 12:59:41+00:00,235,NaN,97.5
3,2019-06-07 13:04:41+00:00,252,NaN,97.5
4,2019-06-07 13:09:41+00:00,257,NaN,97.5
...,...,...,...,...
31561,2019-09-28 10:46:15+00:00,107,NaN,NaN
31562,2019-09-28 10:51:15+00:00,109,NaN,NaN
31563,2019-09-28 10:56:15+00:00,112,NaN,NaN
31564,2019-09-28 11:01:16+00:00,117,NaN,NaN


## Perform 5-minute distribution calculation on insulin data
Insulin bolus: Assuming 5-hour linear absorption, divide values ​​into 5-minute intervals.

Basal insulin: Based on a daily dose of 40 units, distributed into 5-minute intervals, corrected for percent change.

In [18]:
# Calculate insulin absorption (linear distribution over 5 hours, evenly divided every 5 minutes)
absorption_interval = 5 * 60 / 5  # 5 hours = 60 minutes divided into 5 minute intervals
result['insulin_bolus_absorbed'] = result['insulin'] / absorption_interval

# If basal insulin data is available, further distribute
daily_basal_units = 40
basal_insulin_interval = daily_basal_units / (24 * 60 / 5)
result['insulin_basal'] = basal_insulin_interval

## Carbohydrate data distributed to 5 hours: 
Similarly, assuming the absorption time of carbohydrates (carbs) is 5 hours, distributed to every 5-minute interval

In [21]:
result['carbs_absorbed'] = result['carbs'] / absorption_interval

## Handle blank values ​​and non-logical values:
SGV (blood glucose data): padded using average or previous value

In [24]:
result['sgv'].fillna(method='ffill', inplace=True)
result['sgv'].fillna(result['sgv'].mean(), inplace=True)

/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/3996182827.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result['sgv'].fillna(method='ffill', inplace=True)
/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/3996182827.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result['sgv'].fillna(method='ffill', inplace=True)
/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/3996182827.py:2: Fut

In [26]:
result['timestamp'] = result['timestamp'].dt.tz_localize(None)
result

,timestamp,sgv,insulin,carbs,insulin_bolus_absorbed,insulin_basal,carbs_absorbed
0,2019-06-07 12:49:41,230,NaN,97.5,NaN,0.138889,1.625
1,2019-06-07 12:54:41,227,NaN,97.5,NaN,0.138889,1.625
2,2019-06-07 12:59:41,235,NaN,97.5,NaN,0.138889,1.625
3,2019-06-07 13:04:41,252,NaN,97.5,NaN,0.138889,1.625
4,2019-06-07 13:09:41,257,NaN,97.5,NaN,0.138889,1.625
...,...,...,...,...,...,...,...
31561,2019-09-28 10:46:15,107,NaN,NaN,NaN,0.138889,NaN
31562,2019-09-28 10:51:15,109,NaN,NaN,NaN,0.138889,NaN
31563,2019-09-28 10:56:15,112,NaN,NaN,NaN,0.138889,NaN
31564,2019-09-28 11:01:16,117,NaN,NaN,NaN,0.138889,NaN


## Clean all non logical values and add missing values.

In [29]:
# Filter out a reasonable range of data
result['sgv'] = result['sgv'].apply(lambda x: x if 40 <= x <= 400 else None) # SGV reasonable range 40-400
result['insulin'] = result['insulin'].apply(lambda x: x if x >= 0 else None)  # Insulin values ​​cannot be negative
result['carbs'] = result['carbs'].apply(lambda x: x if x >= 0 else None)   # Carbohydrates cannot be negative

# Fill in SGV missing values, first use the previous data to fill (forward fill), and then fill in the remaining empty values ​​(backward fill)
result['sgv'] = result['sgv'].fillna(method='ffill').fillna(method='bfill')
print(result['sgv'].isna().sum())

0


/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/128404343.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result['sgv'] = result['sgv'].fillna(method='ffill').fillna(method='bfill')


In [31]:
result['insulin'].fillna(0, inplace=True)
result['carbs'].fillna(0, inplace=True)

print(result.isna().sum())

timestamp                     0
sgv                           0
insulin                       0
carbs                         0
insulin_bolus_absorbed    28451
insulin_basal                 0
carbs_absorbed            26238
dtype: int64


/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/441910812.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result['insulin'].fillna(0, inplace=True)
/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/441910812.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

In [33]:
result

,timestamp,sgv,insulin,carbs,insulin_bolus_absorbed,insulin_basal,carbs_absorbed
0,2019-06-07 12:49:41,230.0,0.0,97.5,NaN,0.138889,1.625
1,2019-06-07 12:54:41,227.0,0.0,97.5,NaN,0.138889,1.625
2,2019-06-07 12:59:41,235.0,0.0,97.5,NaN,0.138889,1.625
3,2019-06-07 13:04:41,252.0,0.0,97.5,NaN,0.138889,1.625
4,2019-06-07 13:09:41,257.0,0.0,97.5,NaN,0.138889,1.625
...,...,...,...,...,...,...,...
31561,2019-09-28 10:46:15,107.0,0.0,0.0,NaN,0.138889,NaN
31562,2019-09-28 10:51:15,109.0,0.0,0.0,NaN,0.138889,NaN
31563,2019-09-28 10:56:15,112.0,0.0,0.0,NaN,0.138889,NaN
31564,2019-09-28 11:01:16,117.0,0.0,0.0,NaN,0.138889,NaN


## For insulin_bolus_absorbed and carbs_absorbed
Missing values ​​are assigned linearly based on the given "5 hour absorption time" logic.

In [36]:
result = result.reset_index(drop=True)

import numpy as np

def distribute_absorption(data, source_column, target_column, hours=5, interval=5):
    """
    Linearly assign the value of source_column (insulin or carbs) to target_column (absorbed column).
    - hours: total absorption time
    - interval: time step in minutes (default is 5 minutes)
    """
    absorption_steps = (hours * 60) // interval  # Calculate the number of time steps within the absorption time
    data[target_column] = 0  # Initialize target column with zeros

    # Find all rows where the source column has a value > 0
    nonzero_indices = data.index[data[source_column].notnull() & (data[source_column] > 0)]
    
    # Distribute values
    for i in nonzero_indices:
        value = data.loc[i, source_column]
        distributed_value = value / absorption_steps  # Evenly distribute value
        end_idx = min(i + absorption_steps, len(data))  # Prevent going out of bounds
        data.loc[i + 1:end_idx, target_column] += distributed_value  # Vectorized addition

    return data


# Apply linear absorption logic to the insulin_bolus_absorbed and carbs_absorbed fields
result = distribute_absorption(result, source_column='insulin', target_column='insulin_bolus_absorbed')
result = distribute_absorption(result, source_column='carbs', target_column='carbs_absorbed')

/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/200974594.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i + 1:end_idx, target_column] += distributed_value  # Vectorized addition
/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/200974594.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625
 1.625 1.625 1.625 1.625 1

In [38]:
print(result.isna().sum())

print(result.head())

timestamp                 0
sgv                       0
insulin                   0
carbs                     0
insulin_bolus_absorbed    0
insulin_basal             0
carbs_absorbed            0
dtype: int64
            timestamp    sgv  insulin  carbs  insulin_bolus_absorbed  \
0 2019-06-07 12:49:41  230.0      0.0   97.5                     0.0   
1 2019-06-07 12:54:41  227.0      0.0   97.5                     0.0   
2 2019-06-07 12:59:41  235.0      0.0   97.5                     0.0   
3 2019-06-07 13:04:41  252.0      0.0   97.5                     0.0   
4 2019-06-07 13:09:41  257.0      0.0   97.5                     0.0   

   insulin_basal  carbs_absorbed  
0       0.138889           0.000  
1       0.138889           1.625  
2       0.138889           3.250  
3       0.138889           4.875  
4       0.138889           6.500  


In [42]:
print("Insulin Number of rows with missing values ​​or 0:", result['insulin'].isna().sum() + (result['insulin'] == 0).sum())
print("Carbs Number of rows with missing values ​​or 0:", result['carbs'].isna().sum() + (result['carbs'] == 0).sum())


Insulin Number of rows with missing values ​​or 0: 28451
Carbs Number of rows with missing values ​​or 0: 26238


In [44]:
# Fill in the insulin and carbs fields
import numpy as np

result['insulin'].replace(0, np.nan, inplace=True)
result['insulin'] = result['insulin'].interpolate(method='linear', limit_direction='both')
result['insulin'].fillna(result['insulin'].mean(), inplace=True)

result['carbs'].replace(0, np.nan, inplace=True)
result['carbs'] = result['carbs'].interpolate(method='linear', limit_direction='both')
result['carbs'].fillna(result['carbs'].mean(), inplace=True)

/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/3411040670.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result['insulin'].replace(0, np.nan, inplace=True)
/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/3411040670.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

In [46]:
result = result.reset_index(drop=True)

result = distribute_absorption(result, source_column='insulin', target_column='insulin_bolus_absorbed')
result = distribute_absorption(result, source_column='carbs', target_column='carbs_absorbed')

print(result.isna().sum())
print(result.head())

/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/200974594.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05
 0.05 0.05 0.05 0.05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i + 1:end_idx, target_column] += distributed_value  # Vectorized addition
/var/folders/_6/br1hrvkn6x9f7l4sfjq8d0g40000gn/T/ipykernel_19879/200974594.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625 1.625
 1.625 1.625 1.625 1.625 1

timestamp                 0
sgv                       0
insulin                   0
carbs                     0
insulin_bolus_absorbed    0
insulin_basal             0
carbs_absorbed            0
dtype: int64
            timestamp    sgv  insulin  carbs  insulin_bolus_absorbed  \
0 2019-06-07 12:49:41  230.0      3.0   97.5                    0.00   
1 2019-06-07 12:54:41  227.0      3.0   97.5                    0.05   
2 2019-06-07 12:59:41  235.0      3.0   97.5                    0.10   
3 2019-06-07 13:04:41  252.0      3.0   97.5                    0.15   
4 2019-06-07 13:09:41  257.0      3.0   97.5                    0.20   

   insulin_basal  carbs_absorbed  
0       0.138889           0.000  
1       0.138889           1.625  
2       0.138889           3.250  
3       0.138889           4.875  
4       0.138889           6.500  


In [48]:
result


,timestamp,sgv,insulin,carbs,insulin_bolus_absorbed,insulin_basal,carbs_absorbed
0,2019-06-07 12:49:41,230.0,3.0,97.5,0.00,0.138889,0.000
1,2019-06-07 12:54:41,227.0,3.0,97.5,0.05,0.138889,1.625
2,2019-06-07 12:59:41,235.0,3.0,97.5,0.10,0.138889,3.250
3,2019-06-07 13:04:41,252.0,3.0,97.5,0.15,0.138889,4.875
4,2019-06-07 13:09:41,257.0,3.0,97.5,0.20,0.138889,6.500
...,...,...,...,...,...,...,...
31561,2019-09-28 10:46:15,107.0,1.0,20.0,1.00,0.138889,20.000
31562,2019-09-28 10:51:15,109.0,1.0,20.0,1.00,0.138889,20.000
31563,2019-09-28 10:56:15,112.0,1.0,20.0,1.00,0.138889,20.000
31564,2019-09-28 11:01:16,117.0,1.0,20.0,1.00,0.138889,20.000


In [50]:
# Save to CSV
output_path = '/Users/yunruyeh/Desktop/W9_health_project/SGV_dataset.csv'
result.to_csv(output_path, index=False)